In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('fare.csv')

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
#NULLVAlues
df.isnull().sum()

In [ ]:
#EDA???
df.dtypes

In [ ]:
df.describe()

In [ ]:
#Trip duration is in seconds.
df['trip_duration'].describe()

In [ ]:
#MINIMUM = 0 - -----> Canceled Trip
#maximum duration ----> 24hours

In [ ]:
sns.distplot(df['trip_duration'])
plt.show()

In [ ]:
sns.distplot(df['distance_traveled'])
plt.show()

In [ ]:
df['distance_traveled'].describe()

In [ ]:
#analyse where ever trip duration is 0
dff = df[['trip_duration', 'distance_traveled']]
dff1 = dff[dff['trip_duration']==0]
dff1.shape

In [ ]:
dff1

In [ ]:
df.columns

In [ ]:
df['num_of_passengers'].nunique()

In [ ]:
df['num_of_passengers'].unique()

In [ ]:
dff = df[['trip_duration', 'distance_traveled', 'num_of_passengers']]
dff1 = dff[dff['num_of_passengers']==0]
dff1.shape

In [ ]:
dff1

In [ ]:
#Drop all tyhe rows where num_passengers==0 using drop functions
df_index = df[df['num_of_passengers']==0].index
df_index
df = df.drop(index = df_index, axis = 0)


In [ ]:
df.shape

In [ ]:
df.shape

In [ ]:
#dropping using slicing
#dff_trial = df[df['num_of_passengers']!=0]

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
#feature Engineering
df['Total_Fare'] = df['fare'] +df['tip']+df['miscellaneous_fees']

In [ ]:
df

In [ ]:
75.00+24+6.300

In [ ]:
df = df.drop(['fare', 'tip', 'miscellaneous_fees'], axis = 1)
df

In [ ]:
#Correlation
df.corr()
-0.5 to +0.5 wek correlation
-0.65 to -1.0 strong negative correlation
0.65 to 1.0 strong positive correlation

In [ ]:
sns.scatterplot(x = df['distance_traveled'], y = df['Total_Fare'], hue = df['surge_applied'])

In [ ]:
for i in df.columns:
    sns.boxplot(df[i])
    plt.title(i)
    plt.show()

In [ ]:
def outlier_detection(data):
    xx = data.columns
    shapes = data.shape
    
    for i in xx:
        print(f"processing columns {i}")
        print(f"old shape is {shapes}")
        
        data_frame = data[i]
        q1 = data_frame.quantile(0.25)
        q3 = data_frame.quantile(0.75)
        iqr = q3-q1
        lower = q1 - 1.5*iqr
        upper = q3 + 1.5*iqr
        
        outliers = data[(data[i] < lower) | (data[i]>upper)]
        data.drop(index = outliers.index, inplace = True)
        
        print(f'New shape is {data.shape}')
    return data

In [ ]:
fin_df = outlier_detection(df)

In [ ]:
fin_df

In [ ]:
fin_df.corr()

In [ ]:
sns.scatterplot(x= fin_df['distance_traveled'], y = fin_df['Total_Fare'])

In [ ]:
#LINearity
#normality
#multicollinearity
#drop features VIF>5 or VIF>10
#Or Do pCA(No need for now)
#build a model
#from summary do feature selection
#with the selected featurtes build model again and check accuracy

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
#!pip install scikit-learn

In [ ]:
x = df.drop('Total_Fare', axis = 1)
y = df['Total_Fare']
x

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size = 0.2, random_state = 10000)

In [ ]:
x.shape
x.set_options()

In [ ]:
ytrain.shape

In [ ]:
model  = LinearRegression().fit(xtrain, ytrain)

In [ ]:
model.coef_

In [ ]:
predicted = model.predict(xtest)

In [ ]:
predicted

In [ ]:
compare  =pd.DataFrame()
compare['actual'] = ytest
compare['predicted'] = predicted

In [ ]:
compare['error'] = compare['actual']- compare['predicted']
compare

In [ ]:
sum(compare['error'])

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
test_accuracy = r2_score(ytest, predicted)

In [ ]:
test_accuracy

In [ ]:
train_pred = model.predict(xtrain)

In [ ]:
train_accuracy = r2_score(ytrain , train_pred)
train_accuracy

In [ ]:
import statsmodels.api as sm

In [ ]:
pr = sm.OLS(y,x).fit()

In [ ]:
pr.summary()

In [ ]:
#Linearity
x
x.columns

#H0 - No Linear Relationship between the Independent and dependent variable
#Ha - Linear Relationship between the Independent and dependent variable

In [ ]:
from scipy.stats import pearsonr
correlation = []
p_value = []
for columns in x.columns:
    corr,pval = pearsonr(x[columns],y)
    correlation.append(corr)
    p_value.append(pval)
pd.DataFrame({"columns":x.columns,"correlation":correlation,"p_value":p_value})
x["num_of_passengers"].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df["num_of_passengers"].value_counts()

In [ ]:
#Homoscedasticity


In [ ]:
residuals = pr.resid

In [ ]:
aux_mod = sm.OLS(residuals**2,x).fit()

In [ ]:
aux_mod.summary()

In [ ]:
statistics = aux_mod.ess/aux_mod.df_resid
p_value = aux_mod.f_pvalue


In [ ]:
# Plot the residuals to check for heteroscedasticity visually
plt.scatter(pr.fittedvalues, residuals)
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted Values')
plt.axhline(y=0, color='red', linestyle='--')
plt.show()

In [ ]:
# Print the test results
print("Breusch-Pagan Test Results:")
print("--------------------------")
print("Test Statistic:", statistics)
print("p-value:", p_value)
print("Heteroscedasticity is present.") if p_value < 0.05 else print("Homoscedasticity is present.")

In [ ]:
#normality in residuals
from scipy.stats import shapiro,anderson
residuals = pr.resid

In [ ]:
stat_shapiro,p_shapiro = shapiro(residuals)
alpha = 0.05
if p_shapiro >alpha:
    print("residuals are normally distributed")
else:
    print("residuals are not normally distributed")

In [ ]:
result = anderson(residuals)

    

In [ ]:
result

In [ ]:
for i in range(len(result.critical_values)):
    sl,cr_val = result.significance_level[i],result.critical_values[i]
    if result.statistic < cr_val:
        print(f"Residuals are normally distributed at {sl} significance level - H0")
    else:
        print(f"Residuals are not normally distributed at {sl} significance level - H0")
        

In [ ]:
result.significance_level[0]

In [ ]:
from scipy.stats import boxcox
fitted_values,lamda = boxcox(df["Total_Fare"])
y = fitted_values

In [ ]:
y = df[df["Total_Fare"]==0].index
df.drop(index = y,axis = 1,inplace = True)

In [ ]:
x = df.drop("Total_Fare",axis =1,inplace = True)


In [ ]:
df